In [4]:
import cv2
img = cv2.imread("download.png")
img = cv2.resize(img,(640,480))

In [6]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#b1 = cv2.getTrackbarPos('b1','image')
#b2 = cv2.getTrackbarPos('b2','image')
#print(str(b1))
# Convert image to binary
_, bw = cv2.threshold(gray, 234, 255,  cv2.THRESH_BINARY_INV)


# Find all the contours in the thresholded image
contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)



In [7]:
import numpy as np
for c in contours:
    sz = len(c)
    data_countour = np.empty((sz, 2), dtype=np.float64)
    sz = len(c)
    data_countour = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_countour.shape[0]):
        data_countour[i,0] = c[i,0,0]
        data_countour[i,1] = c[i,0,1]
    
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_countour, mean)


In [9]:
mean, eigenvectors, eigenvalues

(array([[120.87725632,  86.92238267]]),
 array([[ 0.78088498, -0.62467484],
        [ 0.62467484,  0.78088498]]),
 array([[3186.53073321],
        [1874.38860954]]))

In [ ]:
data_countour = []
for contour in contours:
    # for i in range(data_countour.shape[0]):
    #     data_countour[i,0] = contour[i,0,0]
    #     data_countour[i,1] = contour[i,0,1]
    data_countour=contour.resize(413,2)

In [19]:
img = cv2.imread("motor1.jpg")
#img = cv2.imread("download.png")
img = cv2.resize(img,(640,480))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, bw = cv2.threshold(gray, 176, 255,  cv2.THRESH_BINARY)
cv2.imshow('bw',bw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)


In [21]:
for i, c in enumerate(contours):
    area = cv2.contourArea(c)

In [23]:
int(area)

0

In [27]:
import numpy as np
import cv2


# Function for returning solidity of contour - ratio of contour area to its 
# convex hull area.
def checkSolidity(cnt):
    area = cv2.contourArea(cnt)
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull)
    solidity = float(area)/hull_area
    return solidity


img_orig = cv2.imread("motor1.jpg")
# Had to dilate the image so the contour was completly connected.
img = cv2.dilate(img_orig, np.ones((3, 3), np.uint8))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Grayscale transformation.
# Otsu threshold.
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
# Search for contours.
contours = cv2.findContours(thresh, cv2.CHAIN_APPROX_NONE, cv2.RETR_TREE)[0]

# Sorting contours from smallest to biggest.
#contours.sort(key=lambda cnt: cv2.contourArea(cnt))

# Three contours - two semi circles and the intersection of the circles.
cnt1 = contours[0]
cnt2 = contours[1]
cnt3 = contours[2]

# Create three empty images
h, w = img.shape[:2]
mask1 = np.zeros((h, w), np.uint8)
mask2 = np.zeros((h, w), np.uint8)
mask3 = np.zeros((h, w), np.uint8)

# Draw all combinations of two out of three contours on the masks.
# The goal here is to draw one semicircle and the intersection together.

cv2.drawContours(mask1, [cnt1], 0, (255, 255, 255), -1)
cv2.drawContours(mask1, [cnt3], 0, (255, 255, 255), -1)

cv2.drawContours(mask2, [cnt2], 0, (255, 255, 255), -1)
cv2.drawContours(mask2, [cnt3], 0, (255, 255, 255), -1)

cv2.drawContours(mask3, [cnt1], 0, (255, 255, 255), -1)
cv2.drawContours(mask3, [cnt2], 0, (255, 255, 255), -1)


# Perform closing operation on the masks so that you get uniform contours.
kernel_size = 25
kernel = np.ones((kernel_size, kernel_size), np.uint8)
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel)
mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel)
mask3 = cv2.morphologyEx(mask3, cv2.MORPH_CLOSE, kernel)

masks = []  # List for storing all the masks.
masks.append(mask1)
masks.append(mask2)
masks.append(mask3)

# List where you will append solidity of the found biggest contour of every mask.
solidity = []
for mask in masks:
    cnts = cv2.findContours(mask, cv2.CHAIN_APPROX_NONE, cv2.RETR_TREE)[0]
    cnt = max(cnts, key=lambda c: cv2.contourArea(c))
    s = checkSolidity(cnt)
    solidity.append(s)


# Index of the mask with smallest solidity.
min_solidity = solidity.index(min(solidity))
# The mask with the contour that has smallest solidity should be the one that
# has two semicirles drawn instead of one semicircle and the intersection. 
#You could build a better function to check which mask is the one with   
# two semicircles... like maybe the contour with the largest 
# height and width of the bounding box etc.
# I chose solidity because it is enough for this example.

# Selection of colors.
colors = {
    0: (0, 0, 255),
    1: (0, 255, 0),
    2: (255, 0, 0),
}

# Draw contours of the mask other two masks - those two that have the        
# semicircle and the intersection.
for i, s in enumerate(solidity):
    if s != solidity[min_solidity]:
        cnts = cv2.findContours(
            masks[i], cv2.CHAIN_APPROX_NONE, cv2.RETR_TREE)[0]
        cnt = max(cnts, key=lambda c: cv2.contourArea(c))
        cv2.drawContours(img_orig, [cnt], 0, colors[i], 1)

# Display result
cv2.imshow("img", img_orig)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:

from __future__ import print_function
 
import cv2 # Import the OpenCV library
import numpy as np # Import Numpy library
import matplotlib.pyplot as plt # Import matplotlib functionality
import sys # Enables the passing of arguments
from common import Sketcher
 
# Project: Image Masking Using OpenCV
# Author: Addison Sears-Collins
# Date created: 9/18/2019
# Python version: 3.7
# Description: This program allows users to highlight a specific 
# object within an image by masking it.
 
# Define the file name of the image
INPUT_IMAGE = "fruits.jpg"
IMAGE_NAME = INPUT_IMAGE[:INPUT_IMAGE.index(".")]
OUTPUT_IMAGE = IMAGE_NAME + "_output.jpg"
TABLE_IMAGE = IMAGE_NAME + "_table.jpg"
 
def main():
    """
    Main method of the program.
    """
    # Pull system arguments
    try:
        fn = sys.argv[1]
    except:
        fn = INPUT_IMAGE
 
    # Load the image and store into a variable
    image = cv2.imread(cv2.samples.findFile(fn))
 
    if image is None:
        print('Failed to load image file:', fn)
        sys.exit(1)
 
    # Create an image for sketching the mask
    image_mark = image.copy()
    sketch = Sketcher('Image', [image_mark], lambda : ((255, 255, 255), 255))
 
    # Sketch a mask
    while True:
        ch = cv2.waitKey()
        if ch == 27: # ESC - exit
            break
        if ch == ord('r'): # r - mask the image
            break
        if ch == ord(' '): # SPACE - reset the inpainting mask
            image_mark[:] = image
            sketch.show()
 
    # define range of white color in HSV
    lower_white = np.array([0,0,255])
    upper_white = np.array([255,255,255])
 
    # Create the mask
    mask = cv2.inRange(image_mark, lower_white, upper_white)
 
    # Create the inverted mask
    mask_inv = cv2.bitwise_not(mask)
 
    # Convert to grayscale image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
    # Extract the dimensions of the original image
    rows, cols, channels = image.shape
    image = image[0:rows, 0:cols]
 
    # Bitwise-OR mask and original image
    colored_portion = cv2.bitwise_or(image, image, mask = mask)
    colored_portion = colored_portion[0:rows, 0:cols]
 
    # Bitwise-OR inverse mask and grayscale image
    gray_portion = cv2.bitwise_or(gray, gray, mask = mask_inv)
    gray_portion = np.stack((gray_portion,)*3, axis=-1)
 
    # Combine the two images
    output = colored_portion + gray_portion
 
    # Save the image
    cv2.imwrite(OUTPUT_IMAGE, output)
 
    # Create a table showing input image, mask, and output
    mask = np.stack((mask,)*3, axis=-1)
    table_of_images = np.concatenate((image, mask, output), axis=1)
    cv2.imwrite(TABLE_IMAGE, table_of_images)
 
    # Display images, used for debugging
    #cv2.imshow('Original Image', image)
    #cv2.imshow('Sketched Mask', image_mark)
    #cv2.imshow('Mask', mask)
    #cv2.imshow('Output Image', output)
    cv2.imshow('Table of Images', table_of_images)
    cv2.waitKey(0) # Wait for a keyboard event
 
if __name__ == '__main__':
    print(__doc__)
    main()
    cv2.destroyAllWindows()

In [7]:
import cv2 
from matplotlib import pyplot as plt
import numpy as np
    
# Read image img
img = cv2.imread('motor1.jpg')
# Binarize
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(img_gray, 176, 255, cv2.THRESH_BINARY)
    
# Find Contours
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
# Draw Contours
blank_mask = np.zeros((thresh.shape[0],thresh.shape[1],3), np.uint8)
cv2.drawContours(blank_mask, contours, -1, (0, 255, 0), 1) 
#cv2.drawContours(thresh, contours, -1, (0, 255, 0), 1) 


a=0
for i, c in enumerate(contours):
        # Calculate the area of each contour
    area = cv2.contourArea(c)
    if int(area) > 1000:
        cv2.putText(img,str(int(area)),(20,a*30), 1,1,(0,0,255),1)
        #a+=1
    x,y,w,h = cv2.boundingRect(c)
    # Ignore contours that are too small or too large
    if area < 8000 or 10000 < area:
        continue

    # Draw each contour only for visualisation purposes
    cv2.drawContours(img, c, -1, (0, 0, 255), 2)
    center = (x+w//2, y+h//2)
    #cv2.circle(img,center,1,(255,0,0),1)
    cv2.line(img,(x+w//2,y),(x+w//2,y+h),(0,255,0),2)
    cv2.putText(img,str(int(w*h)),(400,a*30), 1,1,(0,0,255),1)
    a+=1
    #cv2.bitwise_and(area,)
    #cv2.putText(img,str(area),((x,y)),1,2,(0,0,255),2)
    # Find the orientation of each shape
    #angle = getOrientation(c, img)
    #cv2.putText(img,str(int(angle)),((x,y)),1,2,(0,0,255),2)
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    #print(angle)


    x1= x
    x2= x+w//2
    y1= y
    y2= y+h//2

    # Define lines coordinates
    line1 = [x1, y1, x2, y2]
    line2 = [x1, y1, x2, y2]
    line3 = [x1, y1, x2, y2]
        
    # Draw Lines over Contours
    cv2.line(blank_mask, (line1[0], line1[1]), (line1[2], line1[3]), (255, 0, 0), thickness=30)
    cv2.line(blank_mask, (line2[0], line2[1]), (line2[2], line2[3]), (255, 0, 0), thickness=30)
    cv2.line(blank_mask, (line3[0], line3[1]), (line3[2], line3[3]), (255, 0, 0), thickness=30)
    # Show Image
    cv2.imshow('Output Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#cv2.waitKey(0)

cv2.destroyAllWindows()
# fig = plt.figure(figsize=(50,50))
# ax  = fig.add_subplot(111)
# ax.imshow(blank_mask)
# #ax.imshow(thresh)